# Multiclass SVM 구현

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [65]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [66]:
X_train.shape, X_test.shape

((120, 4), (30, 4))

Y 값을 one hot encoding한다.

In [67]:
y_train =pd.get_dummies(y_train)
y_train

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0
...,...,...,...
64,0,1,0
91,0,1,0
81,0,1,0
51,0,1,0


### One vs Rest 

클래스가 3개이므로 3개의 모델을 만들어준다.

In [68]:
svc1=SVC(kernel='linear') 
svc2=SVC(kernel='linear') 
svc3=SVC(kernel='linear') 

In [69]:
# 선언된 모델에 각각 클래스별로 fit 해준다.

svc1.fit(X_train,y_train.iloc[:,0]) 
svc2.fit(X_train,y_train.iloc[:,1]) 
svc3.fit(X_train,y_train.iloc[:,2]) 

y_pred1=svc1.predict(X_test)
y_pred2=svc2.predict(X_test)
y_pred3=svc3.predict(X_test)


In [70]:
print(y_pred1)
print(y_pred2)
print(y_pred3)

[0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0]


결과를 확인해보니 둘이상 예측되거나 아예 예측되지 않는 경우가 있어 이를 처리하기 위한 알고리즘을 구현한다.

이를 처리하기 위해 Decision_function을 이용하기로 한다.

3개의 모델로 돌린 decision_function 값을 비교하여 가장 높은 점수가 나온쪽으로 분류한다.


In [82]:
models = [svc1,svc2,svc3]
label = y_train.columns
dfm = []
for model in models :
    dfm.append(model.decision_function(X_test))
dfm = np.array(dfm)
result = label[np.argmax(dfm, axis =0)]
result


Index(['versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica',
       'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica',
       'setosa', 'setosa', 'virginica', 'versicolor', 'virginica', 'setosa',
       'versicolor', 'versicolor', 'virginica', 'setosa', 'virginica',
       'virginica', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica',
       'virginica', 'versicolor'],
      dtype='object')

In [83]:
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,result))

Accuracy Score:
0.8333333333333334


잘 예측이 되는 것을 확인할 수 있다.